In [1]:
import numpy as np
from sklearn.neighbors import BallTree

from rangesearch_box import rangesearch_box

# Initialization

In [2]:
ndim = 5
rad = .2
box = [(-rad, rad)] * ndim

In [3]:
X = np.random.rand(500, ndim)
X[:5]

array([[ 0.96248984,  0.86519489,  0.27083233,  0.13151992,  0.6565823 ],
       [ 0.62995518,  0.85749448,  0.71506439,  0.09999372,  0.79918215],
       [ 0.10419382,  0.58455911,  0.79610992,  0.42175123,  0.44563308],
       [ 0.16761207,  0.91066151,  0.26891934,  0.95369074,  0.7298104 ],
       [ 0.81688805,  0.24101873,  0.76913797,  0.89615646,  0.46641426]])

# Compare results

In order to compare the results of the range query, we create here corresponding metric for ball tree.<br>
We can note that we loose the radius property of it.

In [4]:
def box_metric(box):
    def _box_metric(a, b):
        return any([not(box_d_min < b_d - a_d < box_d_max) for a_d, b_d, (box_d_min, box_d_max) in zip(a, b, box)])
    return _box_metric

In [5]:
bt = BallTree(X, metric=box_metric(box))
bt.query_radius(X[(2, 3), :], 0.)

array([array([ 51, 407, 368,   2,  60, 479, 459]), array([369,   3, 260])], dtype=object)

In [6]:
search = rangesearch_box(X)
search(X[(2, 3), :], box)

array([array([  2,  51,  60, 368, 407, 459, 479]), array([  3, 260, 369])], dtype=object)

We find the same results for a given query.

# Compare perf

## Rangesearch init

In [7]:
%timeit rangesearch_box(X)
%timeit BallTree(X, metric=box_metric(box))

1000 loops, best of 3: 483 µs per loop
100 loops, best of 3: 9.95 ms per loop


## Query a few points

In [8]:
%timeit search(X[(2, 6, 10), :], box)
%timeit bt.query_radius(X[(2, 6, 10), :], 0.)

1000 loops, best of 3: 331 µs per loop
100 loops, best of 3: 7.91 ms per loop


## Query more points

In [9]:
y = np.random.rand(500, ndim)

%timeit search(y, box)
%timeit bt.query_radius(y, 0.)

10 loops, best of 3: 38.3 ms per loop
1 loop, best of 3: 1.3 s per loop


# Conclusion

Despite of the cython part in the implementation of balltree (except the metric), <br>
the rangesearch-box algorithm (based of python + numpy) seems to be faster.

However, it is important to note, there is a tradeoff since we are limited in the box shape we can define.